# Diabetic Patient Analysis and Predictions

In [1]:
import pandas as pd
import os
import tensorflow as tf
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

#load the data file into Pandas DataFrame
diabetes = pd.read_csv('data/diabetes.csv')

# Explore the data loaded
print(diabetes.dtypes)
diabetes.head()

label_encoder = preprocessing.LabelEncoder()
diabetes['Outcome'] = label_encoder.fit_transform(
                        diabetes['Outcome'])

# Convert Pandas DataFrame to a numpy vector
np_diabetes = diabetes.to_numpy().astype(float)

# Extract the feature variables (X)
x = np_diabetes[:,1:9]
x

# Extract the target variable (Y), convert to one-hot-encoding
y = np_diabetes[:,8]
y = tf.keras.utils.to_categorical(y, 2)

# Split training and test data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.10)

print("Shape of feature variables :", x_train.shape)
print("Shape of target variables :", y_train.shape)

2022-12-03 20:15:54.875068: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-03 20:16:04.465402: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-03 20:16:04.465490: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-03 20:16:05.327796: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-03 20:16:32.247483: W tensorflow/stream_executor/platform/de

Pregnancies                   int64
Glucose                       int64
BloodPressure                 int64
SkinThickness                 int64
Insulin                       int64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                           int64
Outcome                       int64
dtype: object
Shape of feature variables : (691, 8)
Shape of target variables : (691, 2)


In [2]:
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.regularizers import l2

# Setup Training Parameters
EPOCHS=20
BATCH_SIZE=128
VERBOSE=1
OUTPUT_CLASSES=len(label_encoder.classes_)
N_HIDDEN=256
VALIDATION_SPLIT=0.2

# Create a Keras sequential model
diabetes_model = tf.keras.models.Sequential()


# Add a Dense Layer
diabetes_model.add(keras.layers.Dense(N_HIDDEN,
                                input_shape=(8,),
                                name='Dense-Layer-1',
                                activation='relu'))

# Add a second dense layer
diabetes_model.add(keras.layers.Dense(N_HIDDEN,
                                name='Dense-Layer-2',
                                activation='relu'))

# Add a third dense layer
diabetes_model.add(keras.layers.Dense(N_HIDDEN,
                                name='Dense-Layer-3',
                                activation='relu'))

# Add a fourth dense layer
diabetes_model.add(keras.layers.Dense(N_HIDDEN,
                                name='Dense-Layer-4',
                                activation='relu'))

# # Add a fifth dense layer
# diabetes_model.add(keras.layers.Dense(N_HIDDEN,
#                                 name='Dense-Layer-5',
#                                 activation='relu'))

# # Add a sixth dense layer
# diabetes_model.add(keras.layers.Dense(N_HIDDEN,
#                                 name='Dense-Layer-6',
#                                 activation='relu'))

# Add a softmax layer for categorical prediction
diabetes_model.add(keras.layers.Dense(OUTPUT_CLASSES,
                                name='Final',
                                activation='softmax'))

# Compile the model
diabetes_model.compile(
                        loss='categorical_crossentropy',
                        metrics=['accuracy'])

diabetes_model.summary()

# Build the model
diabetes_model.fit(x_train,
                    y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    verbose=VERBOSE,
                    validation_split=VALIDATION_SPLIT)

# Evaluate the model against the test datasetand print resullts
print("\nEvaluation against Test Datase :\n--------------------------------------")
diabetes_model.evaluate(x_test, y_test)

2022-12-03 20:17:03.737250: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-03 20:17:03.816100: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-03 20:17:03.816518: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Asl): /proc/driver/nvidia/version does not exist
2022-12-03 20:17:04.150771: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Dense-Layer-1 (Dense)       (None, 256)               2304      
                                                                 
 Dense-Layer-2 (Dense)       (None, 256)               65792     
                                                                 
 Dense-Layer-3 (Dense)       (None, 256)               65792     
                                                                 
 Dense-Layer-4 (Dense)       (None, 256)               65792     
                                                                 
 Final (Dense)               (None, 2)                 514       
                                                                 
Total params: 200,194
Trainable params: 200,194
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
5/5 [==============================] - 

[0.579460859298706, 0.6883116960525513]

In [3]:
# Pass individual patient to Predict either he/she have diabetese or not ?
import numpy as np

Pregnancies=10
Glucose=101
BloodPressure=76
SkinThickness=48
Insulin=180
BMI=32.9
DiabetesPedigreeFunction=0.171
Age=63
prediction=np.argmax(diabetes_model.predict([[Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age]]), axis=1)
# print(label_encoder.inverse_transform(prediction)==True)
print(label_encoder.inverse_transform(prediction))

1/1 [==============================] - 0s 342ms/step
[1]
